# Section 3: Converting Between CAR and HEALPix Maps

Pixell is designed to allow users to convert maps between HEALPix format and ndmap format.  This is particularly useful for users interested in comparing maps readily available as HEALPix maps to ACT maps.  The following code demonstrates a simple reprojection from one format to another.

To return to the main notebook or to view a different section use the links in the table of contents.

## Table of contents

 --- 
> [Section 1](Section_1_Introduction.ipynb): Introduction to AdvACT Data Release 4

> [Section 2](Section_2_coadded_maps.ipynb): S08-18 Coadded maps from [Naess et al. (2020)](https://www.overleaf.com/5395816498bsknshgxbmyr) and an Introduction to Pixell

> [Section 3](Section_3_HEALPix_CAR_Conversions.ipynb):  Demonstration of methods to convert maps between HEALPix format and the CAR format.

> [Section 4](Section_4_visualize_objects.ipynb):  Guide to using Pixell to look at objects in the maps and stacking our maps on catalogue positions.

> [Section 5](Section_5_Lensing_maps.ipynb):  Introduction to the Lensing maps from [Darwish et al](https://arxiv.org/pdf/2004.01139.pdf)

> [Section 6](Section_6_delensing_param_shifts.ipynb):  Delensing the CMB powerspectrum and comparing the delensed and lensed parameters from [Han et al. 2020, in preparation]

> [Section 7](Section_7_Component_separated_maps.ipynb):  Guide to the use of the Component Separated maps from [Madhavacheril et al. (2020)](https://arxiv.org/abs/1911.05717)

> [Section 8](Section_8_individual_maps_and_harmonic_analysis.ipynb): Maps of Individual Patches and Harmonic Analysis using CAR maps [Aiola et al, in preparation]

> [Section 9](Section_9_Noise_sims.ipynb): Generating Noise Simulations 

> [Section 10](Section_10_E_B_maps.ipynb):  Look at how to generate E and B maps from I, Q, U maps. 

> [Section 11](Section_11_ACT_likelihood.ipynb): Using the ACT Likelihood code
 
---



## Initialize notebook

For this notebook we want to use parts of notebook 1 and notebook 2, since notebook 2 already runs the first notebook internally we can just run [section 2 notebook](Section_2_coadded_maps.ipynb).

This call will print out the maps produced by the previous notebook so feel free to go back to that notebook if you would like an understanding of how these maps are produced.

In [ ]:
%run Section_2_coadded_maps.ipynb

## Begin Converting Between HEALPix and CAR


When reprojecting it's helpful to keep in mind that Pixell automatically rotates maps from Planck's Galactic projection to Equatorial; if you are projecting a map from other coordinates, a different rotation may be needed. 

Note that projections work by finding the spherical harmonic transformation of the HEALPix map to alm's and then inversely transforms the alm's to the rectangular pixel map geometry.  

Finally for anyone in looking at small patches of the sky we encourage you to use the maps in CAR format with Pixell as it enables users to reference patches in a way that HEALPix can't do.

### Convert a planck HEALPix map to CAR

In [ ]:
# Read in Planck Map
pmap_fname = path + "HFI_SkyMap_143_2048_R2.02_full.fits"

# Reproject the map to the same shape and wcs as our smap_2 from the previous notebook (linked in the table of contents as section 2)
# Use the "unit" argument to convert Planck from Kelvin to uKelvin
planck_map = reproject.enmap_from_healpix(pmap_fname, smap_2.shape, smap_2.wcs,
                                  ncomp=1, unit=1e-6, lmax=6000,rot='gal,equ')

# Visualize using eshow
eshow(planck_map, **{"downgrade": 8, "colorbar":True, "ticks": 10, })

### Convert a CAR map to HEALPix 

We can also take our trimmed ACT map and project it to the HEALPix format using reproject as shown here.

In [ ]:
# Convert an ndmap map to HEALPix
smap_healpix = reproject.healpix_from_enmap(smap_2, lmax = 6000, nside = 512)

# Plot using healpy
lonra = np.sort(smap_2.box()[:, 1])/utils.degree
latra = np.sort(smap_2.box()[:, 0])/utils.degree
rang = 300

hp.cartview(smap_healpix, lonra = lonra, latra = latra, min = -rang, max = rang, 
            cmap = cm.get_cmap('RdYlBu'))

Keep in mind that these reprojection methods bandlimit the result according to the parameter "lmax." See the Pixell documentation for more information.

### Continue to the next section

[Section 4](Section_4_visualize_objects.ipynb):  Guide to using Pixell to look at objects in the maps and stacking our maps on catalogue positions.